In [1]:
#import funcs 
from m6SCRATCH_Funcs import * 

Using TensorFlow backend.


In [2]:
#get m6A sites from multiple tables and pull only those that meet given motif criteria 
m6A_sites = readM6A(['A549_sites_hg19.bed','CD8T_sites_hg19.bed','HeLa_sites_hg19.bed'])
m6A_RAC = verifyMotif('GRCh37.primary_assembly.genome.fa',m6A_sites,'RAC')
m6A_RRACH = verifyMotif('GRCh37.primary_assembly.genome.fa',m6A_sites,'RRACH')

m6A_RAC.to_csv('m6A_RAC.tab',sep = '\t',header = False,index = False)
m6A_RRACH.to_csv('m6A_RRACH.tab',sep = '\t',header = False,index = False)

In [3]:
trxts = processGff('gencode.v19.annotation.gtf')
m6A_peaks = readM6A(['A549_peaks_hg19.bed','CD8T_peaks_hg19.bed','HeLa_peaks_hg19.bed'])
negSites_RRACH = negsOutsidePeaks('GRCh37.primary_assembly.genome.fa', m6A_peaks, m6A_RRACH, trxts,'RRACH')
negSites_RAC = negsOutsidePeaks('GRCh37.primary_assembly.genome.fa', m6A_peaks, m6A_RAC, trxts, 'RAC')

negSites_RAC.to_csv('negSites_RAC.tab',sep = '\t',header = False,index = False)
negSites_RRACH.to_csv('negSites_RRACH.tab',sep = '\t',header = False,index = False)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pybedtools/bedtool.py:3433: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 15 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pybedtools/bedtool.py:3439: FutureWarning: read_table is deprecated, use read_csv instead.
  return pandas.read_table(self.fn, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pybedtools/bedtool.py:3433: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 15 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/Library/Frameworks/Python.framework

In [ ]:
#read files back
negSites=pd.read_csv('negSites_RRACH.tab',sep='\t',comment='#',header=None)
m6A_sites=pd.read_csv('m6A_RRACH.tab',sep='\t',comment='#',header=None)

In [ ]:
#matrix making (Training sets)
trnInpMtxs_POS = getInputMtxs_DNA('GRCh37.primary_assembly.genome.fa',m6A_sites,5000,
                                  [2,3,7,4,6,8,'X','Y']+list(range(10,23)))

trnInpMtxs_NEG = getInputMtxs_DNA('GRCh37.primary_assembly.genome.fa',negSites,5000,
                                  [2,3,7,4,6,8,'X','Y']+list(range(10,23)))

trnInpMtxs = trnInpMtxs_POS+trnInpMtxs_NEG
trnTgtMtxs = [1 for i in range(len(trnInpMtxs_POS))] + [0 for i in range(len(trnInpMtxs_NEG))]

trnInpMtxs,trnTgtMtxs = randSubBlocks(trnInpMtxs,trnTgtMtxs)
Mtxs_toH5Blocks(trnInpMtxs,trnTgtMtxs,100,'train')

In [ ]:
#matrix making (Testing sets)
tstInpMtxs_POS = getInputMtxs_DNA('GRCh37.primary_assembly.genome.fa', m6A_sites, 5000, [1,5,9])

tstInpMtxs_NEG = getInputMtxs_DNA('GRCh37.primary_assembly.genome.fa', negSites, 5000, [1,5,9])

tstInpMtxs = tstInpMtxs_POS+tstInpMtxs_NEG
tstTgtMtxs = [1 for i in range(len(tstInpMtxs_POS))] + [0 for i in range(len(tstInpMtxs_NEG))]

tstInpMtxs,tstTgtMtxs = randSubBlocks(tstInpMtxs,tstTgtMtxs)
Mtxs_toH5Blocks(tstInpMtxs,tstTgtMtxs,100,'test')